##Somalia Road Density
This notebook seeks to look at the distribution of roads across the counties in Somalia.

In [ ]:

!pip install pyshp
!pip install pyproj
!pip install geopy

!sudo apt install libspatialindex-dev
!pip install rtree
!pip install pygeos
!pip install folium
!pip install geopandas

In [ ]:
#import

import pygeos
from shapely.geometry import Point
import geopandas as gpd
import os
import pandas as pd
import numpy as np

import shapefile
import pyproj #https://stackoverflow.com/questions/68317672/coordinate-conversion-script-isnt-giving-me-an-accurate-reading-svy21-to-wgs84
import geopy.distance #https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

import geopandas as gpd
import folium
import branca
import branca.colormap as cm

Boiler plate code

In [ ]:
def makingmap(Dataset,name='road density'):
  Somalia=[8.408416,48.4828]
  map=folium.Map(location=Somalia, zoom_start=6)

  #guide:https://towardsdatascience.com/the-battle-of-choropleths-part-3-folium-86ab1232afc

  #First determine your maximum and minimum values - this becomes the basis for the steps in the map
  vmax = Dataset['Region_road_Density'].max()
  vmin = Dataset['Region_road_Density'].min()
  colormap = cm.LinearColormap(colors=['red','orange','yellow', '#c7f6b6' ,'green'], 
                              vmin=vmin,
                              vmax=vmax,caption= str(name) +' in Somalia')

  n_steps = 7 # Quantiles
  list_of_values = Dataset['Region_road_Density'].sort_values()
  #Remove all 0's as the geopandas version did not count 0 on the count of merging
  list_of_values = [i for i in list_of_values if i != 0]
  length = len(list_of_values)
  index = [list_of_values[int((length/n_steps)*i)] for i in range(n_steps)]
  print(index)
  colormap = colormap.to_step(index=index)
  colormap.add_to(map)
  #Add the colormap as a legend


  # First create a function that would call on these values
  style_function = lambda x: {"weight":1.0, 
                              'color':'brown',
                              'fillColor':colormap(x['properties']['Region_road_Density']), 
                              'fillOpacity':0.3}

  #Save it as an object so you can add it
  Dataset=gpd.GeoDataFrame(Dataset,geometry='geometry')

  for _, r in Dataset.iterrows():
      sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
      geo_j = sim_geo.to_json()
      geo_j = folium.GeoJson(data=Dataset,style_function=style_function,
                            overlay=True)
      geo_j.add_to(map)  

  display(map)
  map.save('/content/'+ name +'.html')

Extracting and converting the necessary Shapefile data into Geodataframes

Somalia_all_roads data link:https://data.humdata.org/dataset/56077ecb-d91d-4836-9431-9c00ab122a02/resource/035c6614-759c-492b-bea3-aaef5e342350/download/somalia_allroads.zip

Somalia regional data link:https://geoportal.icpac.net/layers/geonode%3Asom_admbnda_adm2_undp

In [ ]:
#Turning Somalia_all_road_shp to Geo_Dataframe
Road_shp = gpd.read_file('https://data.humdata.org/dataset/56077ecb-d91d-4836-9431-9c00ab122a02/resource/035c6614-759c-492b-bea3-aaef5e342350/download/somalia_allroads.zip')
Road_shp.geometry=Road_shp.geometry.to_crs(epsg='4326')

#Turning Somalia Regional data into Geo_Dataframe
Region_shp= gpd.read_file('https://geoportal.icpac.net/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Asom_admbnda_adm2_undp&outputFormat=SHAPE-ZIP&srs=EPSG%3A4326&format_options=charset%3AUTF-8')
Region_shp.geometry=Region_shp.geometry.to_crs(epsg='4326')

## Choropleth of road density in Somalia

In [ ]:
#get Area of region
Region_shp['Area']=Region_shp.geometry.area
#Look at which road intersects with what region
Road_within_Region=gpd.sjoin(Road_shp,Region_shp,predicate='intersects')

print(Road_within_Region)
#Do a count
Rcount_within_Region=pd.DataFrame(Road_within_Region,columns=['admin2Name','TYPE'])
Rcount_within_Region.reset_index(inplace=True)
Rcount_within_Region_bytown=Rcount_within_Region.groupby(['admin2Name','TYPE']).count()
Rcount_within_Region_bytown.rename(columns={"index":"Number_of_road"},inplace=True)
Rcount_within_Region_bytown.reset_index(inplace=True)

In [ ]:
#Major road
Major_Road_density=Rcount_within_Region_bytown.loc[(Rcount_within_Region_bytown['TYPE']=='Major road')]
Major_Road_density=pd.merge(Region_shp[['Area','admin2Name','geometry']],Major_Road_density,how='left',on='admin2Name')
Major_Road_density['Number_of_road']=Major_Road_density['Number_of_road'].fillna(0.01)
Major_Road_density['Region_road_Density']=Major_Road_density.apply(lambda x: x.Number_of_road/x.Area,axis=1)

makingmap(Major_Road_density,'Major road density')

In [ ]:
#Secondary road
Secondary_road_density=Rcount_within_Region_bytown.loc[(Rcount_within_Region_bytown['TYPE']=='Secondary road')]
Secondary_road_density=pd.merge(Region_shp[['Area','admin2Name','geometry']],Secondary_road_density,how='left',on='admin2Name')
Secondary_road_density['Number_of_road']=Secondary_road_density['Number_of_road'].fillna(0.01)
Secondary_road_density['Region_road_Density']=Secondary_road_density.apply(lambda x: x.Number_of_road/x.Area,axis=1)

makingmap(Secondary_road_density,'Secondary road density')


In [ ]:
#Track
Track_density=Rcount_within_Region_bytown.loc[(Rcount_within_Region_bytown['TYPE']=='Track')]
Track_density=pd.merge(Region_shp[['Area','admin2Name','geometry']],Track_density,on='admin2Name')
Track_density['Region_road_Density']=Track_density.apply(lambda x: x.Number_of_road/x.Area,axis=1)

makingmap(Track_density,'Track_density')


##Map with Road



In [ ]:
Somalia=[8.408416,48.4828]
map=folium.Map(location=Somalia, zoom_start=6)


#link:https://geopandas.org/en/stable/gallery/polygon_plotting_with_folium.html

#Code to add layers of road
def Addlayer(road='',color='red',weight=1.0):
  for _, r in Road_shp.loc[(Road_shp['TYPE']==road)].iterrows():
      sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
      geo_j = sim_geo.to_json()
      geo_j = folium.GeoJson(data=geo_j,
                            style_function=lambda x: {"weight":weight,'color': color})
      geo_j.add_to(map)
    
##Add county boundaries


#make map      
Somalia=[8.408416,48.4828]
map=folium.Map(location=Somalia, zoom_start=6)
Addlayer('Secondary road','orange',2.0) #add secondary layer
Addlayer('Track','brown',0.5) #Add Track Layer
Addlayer('Major road','blue',3.5) #Add Major road layer

    
display(map)
map.save('roads.html')